In [1]:
import os
from datetime import date

import polars as pl
from dotenv import load_dotenv

from eco_stats import BLSClient
from eco_stats.vintage import scrape_range

_ = load_dotenv()

In [2]:
bls = BLSClient(api_key=os.getenv('BLS_API_KEY'))

In [3]:
qcew = (
    bls
    .get_bulk_data('EN')
)

In [14]:
own_codes = [
    ('0', 'Total Covered'),
    ('8', 'Government'),
    ('1', 'Federal Government'),
    ('2', 'State Government'),
    ('3', 'Local Government'),
    ('5', 'Private'),
]

own_list = [i for i, _ in own_codes]

In [6]:
states = [
    '01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', 
    '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', 
    '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', 
    '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', 
    '53', '54', '55', '56', '72', '78', '79'
]

state_list = ['US000'] + [f'{s}000' for s in states]

In [26]:
industries = [
    ('10', '00', 'Total, all industries', 'national'),
    ('101', '06', 'Goods-producing', 'domain'),
    ('102', '07', 'Service-providing', 'domain'),
    ('1011', '10', 'Natural resources and mining', 'supersector'),
    ('1012', '20', 'Construction', 'supersector'),
    ('1013', '30', 'Manufacturing', 'supersector'),
    ('1021', '40', 'Trade, transportation, and utilities', 'supersector'),
    ('1022', '50', 'Information', 'supersector'),
    ('1023', '55', 'Financial activities', 'supersector'),
    ('1024', '60', 'Professional and business services', 'supersector'),
    ('1025', '65', 'Education and health services', 'supersector'),
    ('1026', '70', 'Leisure and hospitality', 'supersector'),
    ('1027', '80', 'Other services', 'supersector'),
    ('1028', '90', 'Public administration', 'supersector'),
    ('1029', '99', 'Unclassified', 'supersector'),
    ('11', '11', 'Agriculture, forestry, fishing and hunting', 'sector'),
    ('21', '21', 'Mining, quarrying, and oil and gas extraction', 'sector'),
    ('22', '22', 'Utilities', 'sector'),
    ('23', '23', 'Construction', 'sector'),
    ('31', '31', 'Manufacturing', 'sector'),
    ('42', '42', 'Wholesale trade', 'sector'),
    ('44', '44', 'Retail trade', 'sector'),
    ('48', '48', 'Transportation and warehousing', 'sector'),
    ('51', '51', 'Information', 'sector'),
    ('52', '55', 'Finance and insurance', 'sector'),
    ('53', '53', 'Real estate and rental and leasing', 'sector'),
    ('54', '54', 'Professional, scientific, and technical services', 'sector'),
    ('55', '55', 'Management of companies and enterprises', 'sector'),
    ('56', '56', 'Administrative and support and waste management and remediation services', 'sector'),
    ('61', '61', 'Educational services', 'sector'),
    ('62', '62', 'Health care and social assistance', 'sector'),
    ('71', '71', 'Arts, entertainment, and recreation', 'sector'),
    ('72', '72', 'Accommodation and food services', 'sector'),
    ('81', '81', 'Other services (except public administration)', 'sector'),
    ('92', '92', 'Public administration', 'sector'),
    ('99', '99', 'Unclassified', 'sector')
]

industry_list = [i for i, _, _, _ in industries]
industry_dict = {i: c for i, c, _, _ in industries}

domain = [c for _, c, _, t in industries if t == 'domain']
supersector = [c for _, c, _, t in industries if t == 'supersector']
sector = [c for _, c, _, t in industries if t == 'sector']

In [36]:
qcew_all = (
    qcew
    .filter(
        pl.col('area_fips').is_in(state_list),
        pl.col('industry_code').is_in(industry_list),
        pl.col('own_code').is_in(own_list),
        pl.col('size_code').eq('0')
    )
    .with_columns(
        ref_year=pl.col('year')
                  .cast(pl.UInt16),
        ref_month=pl.col('qtr')
                    .cast(pl.UInt8)
                    .sub(1)
                    .mul(3)
    )
    .select(
        ref_date=pl.lit(None, pl.Date),
        ref_year=pl.col('ref_year'),
        ref_month=pl.col('ref_month'),
        revision=pl.lit(None, pl.UInt8),
        vintage_date=pl.lit(None, pl.Date),
        own_code=pl.col('own_code'),
        agglvl_code=pl.col('agglvl_code'),
        geographic_type=pl.lit(None, pl.Utf8),
        geographic_code=pl.col('area_fips')
                          .str.slice(0, 2)
                          .replace({'US': '00'}),
        industry_type=pl.lit(None, pl.Utf8),
        industry_code=pl.col('industry_code'),
        emp_m1=pl.col('month1_emplvl'),
        emp_m2=pl.col('month2_emplvl'),
        emp_m3=pl.col('month3_emplvl'),
    )
    .unpivot(
        ['emp_m1', 'emp_m2', 'emp_m3'],
        index=[
            'ref_date', 'ref_year', 'ref_month', 
            'revision', 'vintage_date',
            'geographic_type', 'geographic_code',
            'industry_type', 'industry_code',
            'own_code', 'agglvl_code'
        ],
        variable_name='month',
        value_name='emp',
    )
    .with_columns(
        month=pl.col('month')
                .str.replace('emp_m', '')
                .cast(pl.UInt8)
    )
    .with_columns(
        ref_month=pl.col('ref_month')
                    .add(pl.col('month'))
    )
    .with_columns(
        ref_date=pl.date(
            pl.col('ref_year'),
            pl.col('ref_month'),
            pl.lit(12, pl.UInt8)
        )
    )
    .filter(
        pl.col('ref_date').eq(pl.date(2016, 1, 12))
    )
)

In [50]:
qcew_national = (
    qcew_all
    .filter(
        pl.col('agglvl_code').is_in(['10', '11', '12', '13'])
    )
    .with_columns(
        geographic_type=pl.lit('national', pl.Utf8),
        industry_type=pl.lit('national', pl.Utf8),
        industry_code=pl.lit('00', pl.Utf8),
    )
    #.drop('own_code', 'agglvl_code', 'month')
)

qcew_national

ref_date,ref_year,ref_month,revision,vintage_date,geographic_type,geographic_code,industry_type,industry_code,own_code,agglvl_code,month,emp
date,u16,u8,u8,date,str,str,str,str,str,str,u8,i64
2016-01-12,2016,1,null,null,"""national""","""00""","""national""","""00""","""0""","""10""",1,138771439
2016-01-12,2016,1,null,null,"""national""","""00""","""national""","""00""","""1""","""11""",1,2755771
2016-01-12,2016,1,null,null,"""national""","""00""","""national""","""00""","""2""","""11""",1,4489129
2016-01-12,2016,1,null,null,"""national""","""00""","""national""","""00""","""3""","""11""",1,14106610
2016-01-12,2016,1,null,null,"""national""","""00""","""national""","""00""","""5""","""11""",1,117419929


In [48]:
qcew_state = (
    qcew_all
    .filter(
        pl.col('agglvl_code').eq('50')
    )
    .with_columns(
        geographic_type=pl.lit('state', pl.Utf8),
        industry_type=pl.lit('national', pl.Utf8),
        industry_code=pl.lit('00', pl.Utf8),
    )
    .drop('own_code', 'agglvl_code', 'month')
)

In [42]:
qcew_gov_national = (
    qcew_all
    .filter(
        pl.col('agglvl_code').eq('95')
    )
    .with_columns(
        geographic_type=pl.lit('national', pl.Utf8),
        industry_type=pl.lit('supersector', pl.Utf8),
        industry_code=pl.lit('90', pl.Utf8),
    )
    .drop('own_code', 'agglvl_code', 'month')
)

In [46]:
qcew_gov_state = (
    qcew_all
    .filter(
        pl.col('agglvl_code').eq('96')
    )
    .with_columns(
        geographic_type=pl.lit('state', pl.Utf8),
        industry_type=pl.lit('supersector', pl.Utf8),
        industry_code=pl.lit('90', pl.Utf8),
    )
    .drop('own_code', 'agglvl_code', 'month')
)

In [ ]:
vintage = (
    scrape_range(
        start_year=2016,
        end_year=2026,
        delay=1.0
    )
    .filter(
        pl.col('source') == 'qcew'
    )
    .select(
        ref_date=pl.col('ref_date'),
        vintage_date=pl.col('release_date')
    )
    .unique(subset=['ref_date', 'vintage_date'], keep='last')
    .sort('ref_date')
    .filter(
        pl.col('ref_date').is_between(
            date(2016, 1, 12),
            date(2026, 1, 12)
        )
    )
)

In [ ]:
vintage

In [ ]:
vintage_list = []
for i in range(1, 6):

    vintage_list.append(
        vintage
        .select(
            ref_date=pl.col('ref_date'),
            revision=pl.lit(i, pl.UInt8),
            vintage_date=pl.col('vintage_date').shift(i)
        )
        .with_columns(
            revision=pl.when(pl.col('vintage_date').is_null())
                       .then(pl.lit(None, pl.UInt8))
                       .otherwise(pl.col('revision'))
        )
        .filter(
            pl.col('vintage_date').is_not_null(),
            pl.col('vintage_date').lt(pl.date(2026, 2, 18))
        )
    )

vintage = (
    pl
    .concat(
        vintage_list
    )
    .filter(
        (pl.col('ref_date').dt.quarter().eq(1) & pl.col('revision').is_in([0, 1, 2, 3, 4])) |
        (pl.col('ref_date').dt.quarter().eq(2) & pl.col('revision').is_in([0, 1, 2, 3])) |
        (pl.col('ref_date').dt.quarter().eq(3) & pl.col('revision').is_in([0, 1, 2])) |
        (pl.col('ref_date').dt.quarter().eq(4) & pl.col('revision').is_in([0, 1]))
    )
    .sort('ref_date')
    .group_by('ref_date', maintain_order=True)
    .agg(
        pl.col('revision').max(),
        pl.col('vintage_date').max()
    )
)
vintage

In [ ]:
vintage_0 = {d[0]: d[1] for d in vintage.iter_rows() if d[1] is not None}
vintage_1 = {d[0]: d[2] for d in vintage.iter_rows() if d[2] is not None}
vintage_2 = {d[0]: d[3] for d in vintage.iter_rows() if d[3] is not None}
vintage_3 = {d[0]: d[4] for d in vintage.iter_rows() if d[4] is not None}
vintage_4 = {d[0]: d[5] for d in vintage.iter_rows() if d[5] is not None}
vintage_5 = {d[0]: d[6] for d in vintage.iter_rows() if d[6] is not None}

In [ ]:
dict_list = []
for d in list(
    vintage
    .iter_rows(named=True)
):

    ref_date = d['ref_date']
    vintage_date_0=d['vintage_date_0']
    vintage_date_1=d['vintage_date_1']

    if ref_date.month == 3:
        r, d = 9, vintage_dict[date(2017, 3, 12)]
    elif date(2016, 4, 12) <= ref_date <= date(2017, 3, 12):
        r, d = 9, vintage_dict[date(2018, 3, 12)]
    elif date(2017, 4, 12) <= ref_date <= date(2018, 3, 12):
        r, d = 9, vintage_dict[date(2019, 3, 12)]
    elif date(2018, 4, 12) <= ref_date <= date(2019, 3, 12):
        r, d = 9, vintage_dict[date(2020, 3, 12)]
    elif date(2019, 4, 12) <= ref_date <= date(2020, 3, 12):
        r, d = 9, vintage_dict[date(2021, 3, 12)]
    elif date(2020, 4, 12) <= ref_date <= date(2021, 3, 12):
        r, d = 9, vintage_dict[date(2022, 3, 12)]
    elif date(2021, 4, 12) <= ref_date <= date(2022, 3, 12):
        r, d = 9, vintage_dict[date(2023, 3, 12)]
    elif date(2022, 4, 12) <= ref_date <= date(2023, 3, 12):
        r, d = 9, vintage_dict[date(2024, 3, 12)]   
    elif date(2023, 4, 12) <= ref_date <= date(2024, 3, 12):
        r, d = 9, vintage_dict[date(2025, 3, 12)]
    elif date(2024, 4, 12) <= ref_date <= date(2025, 3, 12):
        r, d = 9, vintage_dict[date(2026, 1, 12)]
    elif date(2025, 4, 12) <= ref_date < date(2025, 12, 12):
        r, d = 1, vintage_date_1
    else:
        r, d = 0, vintage_date_0

    dict_list.append({
        'ref_date': ref_date,
        'revision': r,
        'vintage_date': d,
    })

revisions = (
    pl
    .DataFrame(
        dict_list,
        schema_overrides={
            'ref_date': pl.Date,
            'revision': pl.UInt8,
            'vintage_date': pl.Date,
        }
    )
)

In [ ]:
states = [
    '01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', 
    '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', 
    '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', 
    '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', 
    '53', '54', '55', '56', '72', '78', '79'
]


adjusted = ['S', 'U']

In [ ]:
industries = [
    ('00000000', '00', 'Total Nonfarm', 'national'),
    
    ('05000000', '05', 'Total Private', 'domain'),
    ('06000000', '06', 'Goods Producing', 'domain'),
    ('07000000', '07', 'Service-Providing', 'domain'),
    ('08000000', '08', 'Private Service Providing', 'domain'),

    ('10000000', '10', 'Mining and Logging', 'supersector'),
    ('20000000', '20', 'Construction', 'supersector'),
    ('30000000', '30', 'Manufacturing', 'supersector'),
    ('40000000', '40', 'Trade, Transportation, and Utilities', 'supersector'),
    ('50000000', '50', 'Information', 'supersector'),
    ('55000000', '55', 'Financial Activities', 'supersector'),
    ('60000000', '60', 'Professional and Business Services', 'supersector'),
    ('65000000', '65', 'Private Education and Health Services', 'supersector'),
    ('70000000', '70', 'Leisure and Hospitality', 'supersector'),
    ('80000000', '80', 'Other Services', 'supersector'),
    ('90000000', '90', 'Government', 'supersector'),

    ('10210000', '21', 'Mining, Quarrying, and Oil and Gas Extraction', 'sector'),
    ('31000000', '31', 'Durable Goods', 'sector'),
    ('32000000', '32', 'Non-Durable Goods', 'sector'),
    ('41000000', '41', 'Wholesale Trade', 'sector'),
    ('42000000', '42', 'Retail Trade', 'sector'),
    ('43220000', '22', 'Utilities', 'sector'),
    ('43400089', '89', 'Transportation and Warehousing', 'sector'),
    ('55520000', '52', 'Finance and Insurance', 'sector'),
    ('55530000', '53', 'Real Estate and Rental and Leasing', 'sector'),
    ('60540000', '54', 'Professional, Scientific, and Technical Services', 'sector'),
    ('60550000', '55', 'Management of Companies and Enterprises', 'sector'),
    ('60560000', '56', 'Administrative and Support and Waste Management and Remediation Services', 'sector'),
    ('65610000', '61', 'Private Educational Services', 'sector'),
    ('65620000', '62', 'Health Care and Social Assistance', 'sector'),
    ('70710000', '71', 'Arts, Entertainment, and Recreation', 'sector'),
    ('70720000', '72', 'Accommodation and Food Services', 'sector'),
    ('90910000', '91', 'Federal Government', 'sector'),
    ('90920000', '92', 'State Government', 'sector'),
    ('90930000', '93', 'Local Government', 'sector'),
]

ind_dict = {i: c for i, c, _, _ in industries}

domain = [c for _, c, _, t in industries if t == 'domain']
supersector = [c for _, c, _, t in industries if t == 'supersector']
sector = [c for _, c, _, t in industries if t == 'sector']

In [ ]:
ces_state_series_ids = []
for a in adjusted:
    for s in states:
        for i in industries:
            ces_state_series_ids.append(f'SM{a}{s}00000{i[0]}01')

len(ces_state_series_ids)

In [ ]:
ces_state = (
    bls
    .get_series(
        series_ids=ces_state_series_ids,
        start_year='2016',
        end_year='2026'
    )
    .with_columns(
        ref_date=pl.col('date')
                   .dt.offset_by('11d'),
        ref_year=pl.col('date')
                   .dt.year()
    )
    .join(
        revisions,
        on='ref_date',
        how='left'
    )
    .select(
        adjusted=pl.col('series_id')
                   .str.slice(2, 1)
                   .eq('S'),
        ref_date=pl.col('ref_date'),
        ref_year=pl.col('ref_year'),
        ref_month=pl.col('ref_date')
                    .dt.month(),
        revision=pl.col('revision'),
        vintage_date=pl.col('vintage_date'),
        geographic_type=pl.lit('state', pl.Utf8),
        geographic_code=pl.col('series_id')
                          .str.slice(3, 2),
        industry_type=pl.lit(None, pl.Utf8),
        industry_code=pl.col('series_id')
                        .str.slice(10, 8)
                        .replace_strict(ind_dict, default=None),
        employment=pl.col('value')
    )
    .with_columns(
        industry_type=pl.when(pl.col('industry_code').eq('00'))
                        .then(pl.lit('national', pl.Utf8))
                        .when(pl.col('industry_code').is_in(domain))
                        .then(pl.lit('domain', pl.Utf8))
                        .when(pl.col('industry_code').is_in(supersector))
                        .then(pl.lit('supersector', pl.Utf8))
                        .when(pl.col('industry_code').is_in(sector))
                        .then(pl.lit('sector', pl.Utf8))
                        .otherwise(pl.lit(None, pl.Utf8))
    )
    .sort(
        'ref_date', 'revision',
        'geographic_type', 'geographic_code',
        'industry_type', 'industry_code'
    )
)

In [ ]:
(
    ces_state
    .filter(
        ~pl.col('adjusted')
    )
    .drop('adjusted')
    .sort(
        'ref_date', 'revision',
        'geographic_type', 'geographic_code',
        'industry_type', 'industry_code'
    )
    .write_parquet(
        '/Users/lowell/Projects/revisions/data/ces_state_nsa.parquet'
    )
)

In [ ]:
(
    ces_state
    .filter(
        pl.col('adjusted')
    )
    .drop('adjusted')
    .sort(
        'ref_date', 'revision',
        'geographic_type', 'geographic_code',
        'industry_type', 'industry_code'
    )
    .write_parquet(
        '/Users/lowell/Projects/revisions/data/ces_state_sa.parquet'
    )
)